In [29]:
import autograd.numpy as np
from autograd import grad

In [30]:
class ORNeuron():
    def __init__(self, n):
        self.n = n
        self.weights = np.zeros(n)
        
    def getWeights(self):
        return self.weights    
    
    def present(self, inputs):
        z = np.sum(np.multiply(inputs, self.weights))
        return 1 - np.power(np.e, -z)
    
    def updateWeights(self, g):
        self.weights -= g
        
        for i in range(0, len(self.weights)):
            if self.weights[i] < 0:
                self.weights[i] = 0
        
    def setGrad(self, g):
        self.grad = g
        
    def applyGrad(self):
        self.updateWeights(self.grad)
        self.grad = None
        
    def __repr__(self):
        return str(self.weights[1:len(self.weights)])

In [31]:
class ANDNeuron():
    def __init__(self, n):
        self.weights = np.zeros(n)
#         self.weights = [1, 0, 1, 0, 1]
        
    def getWeights(self):
        return self.weights
        
    def present(self, inputs):
        i = 1.0 - inputs
        i[0] = 1.0
        
        z = np.sum(np.multiply(i, self.weights))
        return np.power(np.e, -z)
    
    def updateWeights(self, g):
        self.weights -= g
        
        for i in range(0, len(self.weights)):
            if self.weights[i] < 0:
                self.weights[i] = 0
        
    def setGrad(self, g):
        self.grad = g
        
    def applyGrad(self):
        self.updateWeights(self.grad)
        self.grad = None
        
    def __repr__(self):
        return str(self.weights[1:len(self.weights)])

In [32]:
class DNFNetwork():
    def __init__(self, n, c):
        self.n = n
        self.c = c
        
        d = []
        for i in range(0, c):
            d.append(ANDNeuron(n*2 + 1))
        
        self.conjunctions = np.array(d)
        
        if not c == 0:
            self.disjunction = ORNeuron(c + 1)
        else:
            self.disjunction = ORNeuron(n*2 + 1)
        
    def getDisjunction(self):
        return self.disjunction
    
    def getConjunctions(self):
        return self.conjunctions
    
    def fowardprop(self, inputs):
        actualIn = self.__convertInputs__(inputs)
        
        if not self.c == 0:
            dout = [1]
            for d in self.conjunctions:
                dout.append(d.present(actualIn))
            
            actualIn = dout
            
        return self.disjunction.present(np.array(actualIn))
    
    def __convertInputs__(self, inputs):
        actual = [1]
        
        for i in inputs:
            actual.append(i)
            actual.append(1-i)
        
        return np.array(actual)
    
    def __repr__(self):
        s = "Conjunctions -> "
        for d in self.conjunctions:
            s += (str(d) + ", ")
            
        s += ("\nDisjunction -> " + str(self.disjunction) + "\n")
            
        return s

In [33]:
def SSE(network, data, targets, p=False):
    predictions = predict(network, data)
    
#     if p:
#         print(predictions)
    
    return np.sum(np.power(np.subtract(targets, predictions), 2.0))
    
def predict(network, data):
    t = np.array([network.fowardprop(d) for d in data])
    return t
    

In [34]:
def trainDNFNetwork(data, targets, inputNodes, numC, it=10000, lr=0.1):
    network = DNFNetwork(inputNodes, numC)
    print(network)
    print("Initial Loss: ", SSE(network, data, targets, True))
    
    pterb = 0.0001
    
    for i in range(1, it):
        if i%1000 == 0:
            print("Iteration -> " + str(i) + " : " + str(SSE(network, data, targets)))
            
        for d in network.getConjunctions():
            g = computeNeuronGrad(network, d, pterb, data, targets)
            d.setGrad(g * lr)
            
        g = computeNeuronGrad(network, network.getDisjunction(), pterb, data, targets)
        
        network.getDisjunction().updateWeights(g * lr)
        for d in network.getConjunctions():
            d.applyGrad()
            
    print("Trained Loss: ", SSE(network, data, targets, True))
    return network

def computeNeuronGrad(network, neuron, pterb, data, targets):
    gradient = np.zeros(len(neuron.getWeights()))
    for k in range(0, len(neuron.getWeights())):
        g = np.zeros(len(neuron.getWeights()))
        g[k] = -pterb

        oldSSE = SSE(network, data, targets)
        neuron.updateWeights(g)
        newSSE = SSE(network, data, targets)
        neuron.updateWeights(-g)
                
        gradient[k] = (newSSE - oldSSE)/pterb
        
    return gradient

# NOT

In [35]:
dataNOT = np.array([[1.0], [0.0]])
targetsNOT = np.array([0.0, 1.0])
trainDNFNetwork(dataNOT, targetsNOT, 1, 0)

Conjunctions -> 
Disjunction -> [ 0.  0.]

Initial Loss:  1.0
Iteration -> 1000 : 0.00256401191544
Iteration -> 2000 : 0.00126785197053
Iteration -> 3000 : 0.00084175558195
Iteration -> 4000 : 0.00062993425912
Iteration -> 5000 : 0.000503256413748
Iteration -> 6000 : 0.000418984388068
Iteration -> 7000 : 0.000358880936175
Iteration -> 8000 : 0.000313854296033
Iteration -> 9000 : 0.000278864414562
Trained Loss:  0.000250892296604


Conjunctions -> 
Disjunction -> [ 0.         4.1451229]

# AND
For this we dont neeed any disjunctions

In [36]:
dataAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsAND = np.array([0.0, 0.0, 0.0, 1.0])
trainDNFNetwork(dataAND, targetsAND, 2, 1)

Conjunctions -> [ 0.  0.  0.  0.], 
Disjunction -> [ 0.]

Initial Loss:  1.0
Iteration -> 1000 : 0.0150297778406
Iteration -> 2000 : 0.00672034538128
Iteration -> 3000 : 0.00424944234522
Iteration -> 4000 : 0.00308440448997
Iteration -> 5000 : 0.00241143593159
Iteration -> 6000 : 0.00197491656694
Iteration -> 7000 : 0.00166962036877
Iteration -> 8000 : 0.00144449093601
Iteration -> 9000 : 0.00127182713277
Trained Loss:  0.00113532699449


Conjunctions -> [ 5.32906187  0.          5.32906187  0.        ], 
Disjunction -> [ 3.8528426]

# OR

In [37]:
dataOR = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsOR = np.array([0.0, 1.0, 1.0, 1.0])
trainDNFNetwork(dataOR, targetsOR, 2, 0)

Conjunctions -> 
Disjunction -> [ 0.  0.  0.  0.]

Initial Loss:  3.0
Iteration -> 1000 : 0.00521300404887
Iteration -> 2000 : 0.00255871843019
Iteration -> 3000 : 0.00169422754186
Iteration -> 4000 : 0.00126609639561
Iteration -> 5000 : 0.00101059935924
Iteration -> 6000 : 0.000840864237408
Iteration -> 7000 : 0.000719925065581
Iteration -> 8000 : 0.000629388630652
Iteration -> 9000 : 0.000559072863644
Trained Loss:  0.000502885248704


Conjunctions -> 
Disjunction -> [ 4.1439617  0.         4.1439617  0.       ]

# NAND

In [38]:
dataNAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsNAND = np.array([1.0, 1.0, 1.0, 0.0])
trainDNFNetwork(dataNAND, targetsNAND, 2, 2)

Conjunctions -> [ 0.  0.  0.  0.], [ 0.  0.  0.  0.], 
Disjunction -> [ 0.  0.]

Initial Loss:  3.0
Iteration -> 1000 : 0.164350645679
Iteration -> 2000 : 0.128286733334
Iteration -> 3000 : 0.111067878116
Iteration -> 4000 : 0.100276907207
Iteration -> 5000 : 0.0870380065907
Iteration -> 6000 : 0.0163834866093
Iteration -> 7000 : 0.00638441480339
Iteration -> 8000 : 0.00377999841242
Iteration -> 9000 : 0.0026421867324
Trained Loss:  0.00201543311055


Conjunctions -> [ 0.          5.32892233  0.          0.59882425], [ 0.          0.59882425  0.          5.32892233], 
Disjunction -> [ 7.58087306  7.58087306]

# NOR

In [39]:
dataNAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsNAND = np.array([1.0, 0.0, 0.0, 0.0])
trainDNFNetwork(dataNAND, targetsNAND, 2, 1)

Conjunctions -> [ 0.  0.  0.  0.], 
Disjunction -> [ 0.]

Initial Loss:  1.0
Iteration -> 1000 : 0.0150297778406
Iteration -> 2000 : 0.00672034538128
Iteration -> 3000 : 0.00424944234522
Iteration -> 4000 : 0.00308440448997
Iteration -> 5000 : 0.00241143593159
Iteration -> 6000 : 0.00197491656694
Iteration -> 7000 : 0.00166962036877
Iteration -> 8000 : 0.00144449093602
Iteration -> 9000 : 0.00127182713277
Trained Loss:  0.00113532699449


Conjunctions -> [ 0.          5.32906187  0.          5.32906187], 
Disjunction -> [ 3.8528426]

# XOR

In [40]:
dataNAND = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsNAND = np.array([0.0, 1.0, 1.0, 0.0])
trainDNFNetwork(dataNAND, targetsNAND, 2, 2)

Conjunctions -> [ 0.  0.  0.  0.], [ 0.  0.  0.  0.], 
Disjunction -> [ 0.  0.]

Initial Loss:  2.0
Iteration -> 1000 : 0.999999889639
Iteration -> 2000 : 0.0625126009626
Iteration -> 3000 : 0.0183741996254
Iteration -> 4000 : 0.0103801964463
Iteration -> 5000 : 0.00715037599201
Iteration -> 6000 : 0.00542385175814
Iteration -> 7000 : 0.00435535281373
Iteration -> 8000 : 0.00363133748981
Iteration -> 9000 : 0.00310947961787
Trained Loss:  0.00271608129924


Conjunctions -> [ 0.          5.55704148  5.55704148  0.        ], [ 5.55704148  0.          0.          5.55704148], 
Disjunction -> [ 3.76405288  3.76405288]

# IMPLIES

In [41]:
dataIMPLYS = np.array([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
targetsIMPLYS = np.array([1.0, 1.0, 0.0, 1.0])
trainDNFNetwork(dataIMPLYS, targetsIMPLYS, 2, 2)

Conjunctions -> [ 0.  0.  0.  0.], [ 0.  0.  0.  0.], 
Disjunction -> [ 0.  0.]

Initial Loss:  3.0
Iteration -> 1000 : 0.164350645679
Iteration -> 2000 : 0.128286733334
Iteration -> 3000 : 0.111067878204
Iteration -> 4000 : 0.100277818613
Iteration -> 5000 : 0.0913130843561
Iteration -> 6000 : 0.0216573128666
Iteration -> 7000 : 0.00727503328172
Iteration -> 8000 : 0.00410341627637
Iteration -> 9000 : 0.00280380071697
Trained Loss:  0.00211104083687


Conjunctions -> [ 0.          0.61841629  5.29945371  0.        ], [ 0.          5.29945371  0.61841629  0.        ], 
Disjunction -> [ 7.6795138  7.6795138]

# 3OR

In [48]:
data3OR = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targets3OR = np.array([0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
trainDNFNetwork(data3OR, targets3OR, 3, 3)

Conjunctions -> [ 0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.], 
Disjunction -> [ 0.  0.  0.]

Initial Loss:  7.0
Iteration -> 1000 : 0.347514605059
Iteration -> 2000 : 0.303266828533
Iteration -> 3000 : 0.280104610647
Iteration -> 4000 : 0.264657474325
Iteration -> 5000 : 0.253193610462
Iteration -> 6000 : 0.244106207691
Iteration -> 7000 : 0.189533870038
Iteration -> 8000 : 0.0253144617015
Iteration -> 9000 : 0.009785465909
Trained Loss:  0.00576828501625


Conjunctions -> [ 5.05667924  0.          0.74821285  0.          0.          0.        ], [ 0.71809035  0.          0.          0.          5.1584332   0.        ], [ 0.53619824  0.          5.10502443  0.          0.17811326  0.        ], 
Disjunction -> [ 7.88057271  7.82531797  7.80402064]

# 3AND

In [49]:
data3AND = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targets3AND = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0])
trainDNFNetwork(data3AND, targets3AND, 3, 1)

Conjunctions -> [ 0.  0.  0.  0.  0.  0.], 
Disjunction -> [ 0.]

Initial Loss:  1.0
Iteration -> 1000 : 0.0222633324774
Iteration -> 2000 : 0.00976943121523
Iteration -> 3000 : 0.00612037538354
Iteration -> 4000 : 0.00441688595444
Iteration -> 5000 : 0.00343932422994
Iteration -> 6000 : 0.00280824022053
Iteration -> 7000 : 0.00236848025542
Iteration -> 8000 : 0.00204514531646
Iteration -> 9000 : 0.00179776246315
Trained Loss:  0.00160259270776


Conjunctions -> [ 5.29247297  0.          5.29247297  0.          5.29247297  0.        ], 
Disjunction -> [ 3.74587818]

# Complicated Formula

## (a AND b) OR (NOT c)

In [42]:
dataInteresting1 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting1 = np.array([1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0])
trainDNFNetwork(dataInteresting1, targetsInteresting1, 3, 2)

Conjunctions -> [ 0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.], 
Disjunction -> [ 0.  0.]

Initial Loss:  5.0
Iteration -> 1000 : 0.42630707402
Iteration -> 2000 : 0.119632613716
Iteration -> 3000 : 0.014087505093
Iteration -> 4000 : 0.00693242629622
Iteration -> 5000 : 0.00450968869673
Iteration -> 6000 : 0.00331416388312
Iteration -> 7000 : 0.00260792624113
Iteration -> 8000 : 0.00214385539838
Iteration -> 9000 : 0.0018166123475
Trained Loss:  0.00157395738292


Conjunctions -> [ 5.46299493  0.          5.46299493  0.          0.          0.36430708], [  4.77889756e-03   1.38788326e-01   4.77889756e-03   1.38788326e-01
   0.00000000e+00   6.58409320e+00], 
Disjunction -> [ 6.04261969  5.80947776]

## NOT ( a AND b AND c)

In [43]:
dataInteresting2 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting2 = np.array([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0])
trainDNFNetwork(dataInteresting2, targetsInteresting2, 3, 1)

Conjunctions -> [ 0.  0.  0.  0.  0.  0.], 
Disjunction -> [ 0.]

Initial Loss:  7.0
Iteration -> 1000 : 0.41906699634
Iteration -> 2000 : 0.366342146216
Iteration -> 3000 : 0.339874574363
Iteration -> 4000 : 0.322353722209
Iteration -> 5000 : 0.309344710032
Iteration -> 6000 : 0.299049645457
Iteration -> 7000 : 0.290563058046
Iteration -> 8000 : 0.283365510247
Iteration -> 9000 : 0.277131735829
Trained Loss:  0.27164476439


Conjunctions -> [ 0.          1.16992534  0.          1.16992534  0.          1.16992534], 
Disjunction -> [ 17.0855916]

## a AND b AND c

In [44]:
dataInteresting3 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting3 = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0])
trainDNFNetwork(dataInteresting3, targetsInteresting3, 3, 1)

Conjunctions -> [ 0.  0.  0.  0.  0.  0.], 
Disjunction -> [ 0.]

Initial Loss:  1.0
Iteration -> 1000 : 0.0222633324774
Iteration -> 2000 : 0.00976943121523
Iteration -> 3000 : 0.00612037538354
Iteration -> 4000 : 0.00441688595444
Iteration -> 5000 : 0.00343932422994
Iteration -> 6000 : 0.00280824022053
Iteration -> 7000 : 0.00236848025542
Iteration -> 8000 : 0.00204514531646
Iteration -> 9000 : 0.00179776246315
Trained Loss:  0.00160259270776


Conjunctions -> [ 5.29247297  0.          5.29247297  0.          5.29247297  0.        ], 
Disjunction -> [ 3.74587818]

# (a XOR b) AND c
CNF: ((NOT a) OR (NOT b)) AND (a OR b) AND c

In [46]:
dataInteresting4 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting4 = np.array([0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0])
trainDNFNetwork(dataInteresting4, targetsInteresting4, 3, 2, 10000, 0.5)

Conjunctions -> [ 0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.], 
Disjunction -> [ 0.  0.]

Initial Loss:  2.0
Iteration -> 1000 : 1.00106361686
Iteration -> 2000 : 1.00051971505
Iteration -> 3000 : 1.00034336398
Iteration -> 4000 : 1.00025623508
Iteration -> 5000 : 1.00020432145
Iteration -> 6000 : 1.00016987399
Iteration -> 7000 : 1.00014535215
Iteration -> 8000 : 1.00012700858
Iteration -> 9000 : 1.00011277085
Trained Loss:  1.00010139992


Conjunctions -> [ 0.37405018  0.37405018  0.37405018  0.37405018  4.92222613  0.        ], [ 0.37405018  0.37405018  0.37405018  0.37405018  4.92222613  0.        ], 
Disjunction -> [ 1.54704704  1.54704704]

# (a XOR b) OR c
DNF: (a AND (NOT b)) OR ((NOT a) AND b) OR c

In [47]:
dataInteresting5 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting5 = np.array([0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0])
trainDNFNetwork(dataInteresting5, targetsInteresting5, 3, 3)

Conjunctions -> [ 0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.], 
Disjunction -> [ 0.  0.  0.]

Initial Loss:  6.0
Iteration -> 1000 : 1.00144549847
Iteration -> 2000 : 1.00067848541


KeyboardInterrupt: 

# (a XOR b) In 3Space
DNF: (a AND (NOT b)) OR ((NOT a) AND b)

In [50]:
dataInteresting4 = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsInteresting4 = np.array([0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0])
trainDNFNetwork(dataInteresting4, targetsInteresting4, 3, 2)

Conjunctions -> [ 0.  0.  0.  0.  0.  0.], [ 0.  0.  0.  0.  0.  0.], 
Disjunction -> [ 0.  0.]

Initial Loss:  4.0
Iteration -> 1000 : 2.00000000239
Iteration -> 2000 : 2.00000000237
Iteration -> 3000 : 2.00000000235
Iteration -> 4000 : 2.00000000233
Iteration -> 5000 : 2.00000000231
Iteration -> 6000 : 2.00000000229
Iteration -> 7000 : 2.00000000227
Iteration -> 8000 : 2.00000000225
Iteration -> 9000 : 2.00000000224
Trained Loss:  2.00000000222


Conjunctions -> [ 0.10351258  0.10351258  0.10351258  0.10351258  0.10351258  0.10351258], [ 0.10351258  0.10351258  0.10351258  0.10351258  0.10351258  0.10351258], 
Disjunction -> [ 0.41802977  0.41802977]

# (a OR b) in 3Space

In [51]:
dataOR3Space = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0], [0.0, 1.0, 1.0], [1.0, 0.0, 0.0], [1.0, 0.0, 1.0], [1.0, 1.0, 0.0], [1.0, 1.0, 1.0]])
targetsOR3Space = np.array([0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
trainDNFNetwork(dataInteresting4, targetsInteresting4, 3, 1)

Conjunctions -> [ 0.  0.  0.  0.  0.  0.], 
Disjunction -> [ 0.]

Initial Loss:  4.0
Iteration -> 1000 : 2.00000000267
Iteration -> 2000 : 2.00000000264
Iteration -> 3000 : 2.00000000261
Iteration -> 4000 : 2.00000000257
Iteration -> 5000 : 2.00000000254
Iteration -> 6000 : 2.00000000251
Iteration -> 7000 : 2.00000000248
Iteration -> 8000 : 2.00000000245
Iteration -> 9000 : 2.00000000242
Trained Loss:  2.00000000239


Conjunctions -> [ 0.07299454  0.07299454  0.07299454  0.07299454  0.07299454  0.07299454], 
Disjunction -> [ 0.49811337]